In [1]:
import findspark
findspark.init()
findspark.find()

'C:\\work\\spark-2.4.8-bin-hadoop2.7'

In [2]:
from pyspark.sql import SparkSession, Row
spark=SparkSession.builder.appName("SparkSQL").getOrCreate()
spark.version

'2.4.8'

In [25]:
from pyspark.sql.functions import desc

In [3]:
def mapper(line):
    field=line.split(',')
    return Row(ID=int(field[0]), name=str(field[1].encode("utf-8")),
              age=int(field[2]), numFriends=int(field[3]))

In [4]:
lines=spark.sparkContext.textFile("fakefriends.csv")

In [17]:
people=lines.map(mapper)
people.take(5)

[Row(ID=0, age=33, name="b'Will'", numFriends=385),
 Row(ID=1, age=26, name="b'Jean-Luc'", numFriends=2),
 Row(ID=2, age=55, name="b'Hugh'", numFriends=221),
 Row(ID=3, age=40, name="b'Deanna'", numFriends=465),
 Row(ID=4, age=68, name="b'Quark'", numFriends=21)]

In [18]:
schemaPeople=spark.createDataFrame(people).cache()
schemaPeople.show(5)

+---+---+-----------+----------+
| ID|age|       name|numFriends|
+---+---+-----------+----------+
|  0| 33|    b'Will'|       385|
|  1| 26|b'Jean-Luc'|         2|
|  2| 55|    b'Hugh'|       221|
|  3| 40|  b'Deanna'|       465|
|  4| 68|   b'Quark'|        21|
+---+---+-----------+----------+
only showing top 5 rows



In [11]:
schemaPeople.createOrReplaceTempView("people_table")

In [28]:
teen=spark.sql("SELECT * FROM people_table WHERE age >= 13 AND age <= 19")
teen.show(5)

+---+---+----------+----------+
| ID|age|      name|numFriends|
+---+---+----------+----------+
| 21| 19|  b'Miles'|       268|
| 52| 19|b'Beverly'|       269|
| 54| 19|  b'Brunt'|         5|
|106| 18|b'Beverly'|       499|
|115| 18|  b'Dukat'|       397|
+---+---+----------+----------+
only showing top 5 rows



In [27]:
schemaPeople.groupBy("age").count().orderBy(desc("age")).show(5)

+---+-----+
|age|count|
+---+-----+
| 69|   10|
| 68|   10|
| 67|   16|
| 66|    9|
| 65|    5|
+---+-----+
only showing top 5 rows

